# Explore here

In [63]:
# Your code here
#libraris
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import MinMaxScaler
#from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import joblib


In [38]:
#Lectura de datos
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep = ";")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [40]:
#Filtrado de datos
data = data.drop_duplicates().reset_index(drop = True)
data.nunique()

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [41]:
#transformar los datos de objeto para podere tratarlos a lo largo del algorimo
# Crear el encoder
label_encoder = LabelEncoder()

# Aplicar el encoder a la columna
data['job_n'] = label_encoder.fit_transform(data['job'])
data['marital_n'] = label_encoder.fit_transform(data['marital'])
data['education_n'] = label_encoder.fit_transform(data['education'])
data['default_n'] = label_encoder.fit_transform(data['default'])
data['housing_n'] = label_encoder.fit_transform(data['housing'])
data['loan_n'] = label_encoder.fit_transform(data['loan'])
data['contact_n'] = label_encoder.fit_transform(data['contact'])
data['month_n'] = label_encoder.fit_transform(data['month'])
data['day_of_week_n'] = label_encoder.fit_transform(data['day_of_week'])
data['poutcome_n'] = label_encoder.fit_transform(data['poutcome'])
data['y_n'] = label_encoder.fit_transform(data['y'])

data.drop(columns=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome','y'], inplace = True)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41176 entries, 0 to 41175
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   duration        41176 non-null  int64  
 2   campaign        41176 non-null  int64  
 3   pdays           41176 non-null  int64  
 4   previous        41176 non-null  int64  
 5   emp.var.rate    41176 non-null  float64
 6   cons.price.idx  41176 non-null  float64
 7   cons.conf.idx   41176 non-null  float64
 8   euribor3m       41176 non-null  float64
 9   nr.employed     41176 non-null  float64
 10  job_n           41176 non-null  int64  
 11  marital_n       41176 non-null  int64  
 12  education_n     41176 non-null  int64  
 13  default_n       41176 non-null  int64  
 14  housing_n       41176 non-null  int64  
 15  loan_n          41176 non-null  int64  
 16  contact_n       41176 non-null  int64  
 17  month_n         41176 non-null 

In [42]:
#Dividiremos los datos en entrenamiento y test
X = data.drop("y_n", axis = 1)
y = data["y_n"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


In [43]:
# Usamos xgb para conocer la importancia de las características
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42).fit(X_train, y_train)
data_imp = pd.DataFrame({'Feature':X_train.columns, 'Importance':xgb_model.feature_importances_*100})
data_imp = data_imp.sort_values(by='Importance', ascending=False)
data_imp


,Feature,Importance
9,nr.employed,67.783295
1,duration,4.361413
7,cons.conf.idx,3.215527
19,poutcome_n,2.674913
3,pdays,2.302261
6,cons.price.idx,1.952860
17,month_n,1.906862
16,contact_n,1.708631
8,euribor3m,1.553891
13,default_n,1.482968


In [44]:
#Eliminaremos los datos con una importancia menor al 1%
data_imp[data_imp.Importance>=1].Feature.values

X_train = X_train[data_imp[data_imp.Importance>=1].Feature.values]
X_test = X_test[data_imp[data_imp.Importance>=1].Feature.values]


In [45]:
#Para verlo con mayor pespectiva se realizara una mtriz de correlaciones
X_corr = X_train.copy()
X_corr['y'] = y_train
corr = X_corr.corr()
corr.style.background_gradient(cmap='coolwarm').format(precision=3)

,nr.employed,duration,cons.conf.idx,poutcome_n,pdays,cons.price.idx,month_n,contact_n,euribor3m,default_n,day_of_week_n,education_n,job_n,campaign,loan_n,marital_n,housing_n,y
nr.employed,1.000,-0.044,0.098,0.115,0.377,0.519,-0.225,0.269,0.945,0.189,0.028,-0.041,-0.019,0.144,0.007,-0.088,-0.046,-0.358
duration,-0.044,1.000,-0.008,0.033,-0.049,0.006,0.006,-0.026,-0.032,-0.008,0.019,-0.018,-0.004,-0.073,-0.001,0.009,-0.007,0.405
cons.conf.idx,0.098,-0.008,1.000,0.179,-0.097,0.057,0.007,0.248,0.275,0.023,0.045,0.081,0.051,-0.015,-0.009,-0.034,-0.034,0.060
poutcome_n,0.115,0.033,0.179,1.000,-0.478,0.208,-0.064,0.117,0.180,0.022,0.016,0.016,0.014,0.034,-0.000,0.000,-0.010,0.133
pdays,0.377,-0.049,-0.097,-0.478,1.000,0.080,-0.050,0.118,0.300,0.080,-0.009,-0.046,-0.030,0.051,-0.000,-0.037,-0.013,-0.327
cons.price.idx,0.519,0.006,0.057,0.208,0.080,1.000,-0.007,0.591,0.686,0.166,0.005,-0.081,-0.015,0.127,-0.001,-0.057,-0.082,-0.136
month_n,-0.225,0.006,0.007,-0.064,-0.050,-0.007,1.000,0.275,-0.122,-0.017,0.029,-0.082,-0.034,-0.065,-0.004,-0.008,-0.022,-0.001
contact_n,0.269,-0.026,0.248,0.117,0.118,0.591,0.275,1.000,0.399,0.133,-0.011,-0.104,-0.027,0.076,-0.006,-0.055,-0.084,-0.143
euribor3m,0.945,-0.032,0.275,0.180,0.300,0.686,-0.122,0.399,1.000,0.194,0.039,-0.036,-0.007,0.134,0.003,-0.093,-0.061,-0.309
default_n,0.189,-0.008,0.023,0.022,0.080,0.166,-0.017,0.133,0.194,1.000,-0.008,-0.185,-0.030,0.031,-0.001,-0.080,-0.014,-0.097


In [46]:
binary_cols = X_train[(X_train==0)|(X_train==1)].dropna(axis=1).columns.values
binary_cols

array(['contact_n'], dtype=object)

In [47]:
X_train.drop(binary_cols, axis=1).skew()

nr.employed      -1.048155
duration          3.273248
cons.conf.idx     0.303506
poutcome_n       -0.867942
pdays            -4.892273
cons.price.idx   -0.229509
month_n          -0.307360
euribor3m        -0.712231
default_n         1.441344
day_of_week_n     0.008691
education_n      -0.233600
job_n             0.452276
campaign          4.795464
loan_n            1.812362
marital_n        -0.059580
housing_n        -0.148364
dtype: float64

In [49]:
#Escalar las variables no binarias
X_train_num = X_train.drop(binary_cols, axis=1)
X_test_num = X_test.drop(binary_cols, axis=1)

In [52]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_sc = scaler.transform(X_train_num)
X_test_num_sc = scaler.transform(X_test_num)

In [53]:
#union de los datos ecalados con los datos binarios
X_train_sc = pd.concat([pd.DataFrame(X_train_num_sc, columns=X_train_num.columns), X_train[binary_cols].reset_index(drop=True)], axis=1)
X_test_sc = pd.concat([pd.DataFrame(X_test_num_sc, columns=X_test_num.columns), X_test[binary_cols].reset_index(drop=True)], axis=1)

In [54]:
X_train_sc

,nr.employed,duration,cons.conf.idx,poutcome_n,pdays,cons.price.idx,month_n,euribor3m,default_n,day_of_week_n,education_n,job_n,campaign,loan_n,marital_n,housing_n,contact_n
0,1.000000,0.023993,0.338912,0.5,1.0,0.669135,0.333333,0.980730,0.0,0.25,0.428571,0.272727,0.000000,1.0,0.333333,1.0,0
1,0.512287,0.006913,0.192469,0.5,1.0,0.269680,0.666667,0.138291,0.0,0.25,0.714286,0.909091,0.109091,0.0,0.333333,1.0,0
2,0.859735,0.127288,0.602510,0.5,1.0,0.698753,0.666667,0.956926,0.0,0.50,0.714286,0.545455,0.090909,0.0,0.333333,0.0,1
3,0.859735,0.051444,0.602510,0.5,1.0,0.698753,0.666667,0.957379,0.0,0.25,0.428571,0.000000,0.000000,0.0,0.333333,0.0,1
4,0.512287,0.015453,0.154812,0.5,1.0,0.340608,0.000000,0.175924,0.0,0.50,0.857143,0.727273,0.054545,0.0,0.666667,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37053,0.859735,0.020333,0.602510,0.5,1.0,0.698753,0.666667,0.957379,0.0,0.75,0.285714,0.090909,0.018182,1.0,0.333333,0.0,1
37054,1.000000,0.024400,0.376569,0.5,1.0,0.882307,0.444444,0.980957,0.0,0.50,0.285714,0.000000,0.000000,0.0,0.333333,1.0,1
37055,0.203781,0.050834,1.000000,0.0,1.0,0.089634,0.888889,0.027205,0.0,0.50,0.000000,0.454545,0.018182,0.0,0.333333,0.0,0
37056,0.859735,0.059984,0.602510,0.5,1.0,0.698753,0.666667,0.957153,0.0,1.00,0.857143,0.363636,0.018182,0.0,0.333333,1.0,1


In [64]:
#Creamos del estimador
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train_sc, X_test_sc, y_train, y_test)

estimator = ExtraTreesRegressor()

# Definimos una grilla de hiperparámetros para muestrear
hp_distribution = {'n_estimators':np.arange(10,100,10), 'max_depth':np.arange(10,100,10)}

# Realizamos la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator, param_distributions = hp_distribution, n_iter=5, cv=5, random_state=42, n_jobs=-1, scoring='neg_root_mean_squared_error', verbose=2)
random_search.fit(X_train_sc, y_train)

NameError: name 'LazyRegressor' is not defined